<h1 style="text-align:center"> INFO 323: Cloud Computing and Big Data</h1>
<h2 style="text-align:center"> College of Computing and Informatics</h2>
<h2 style="text-align:center">Drexel University</h2>

<h3 style="text-align:center"> Spark Reader and Writer</h3>
<h3 style="text-align:center"> Yuan An, PhD</h3>
<h3 style="text-align:center">Associate Professor</h3>

# Reader & Writer
1. Read from CSV files
1. Read from JSON files
1. Write DataFrame to files
1. Write DataFrame to tables

##### Methods
- DataFrameReader (<a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=dataframereader#pyspark.sql.DataFrameReader" target="_blank">Python</a>/<a href="http://spark.apache.org/docs/latest/api/scala/org/apache/spark/sql/DataFrameReader.html" target="_blank">Scala</a>): `csv`, `json`, `option`, `schema`
- DataFrameWriter (<a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=dataframereader#pyspark.sql.DataFrameWriter" target="_blank">Python</a>/<a href="http://spark.apache.org/docs/latest/api/scala/org/apache/spark/sql/DataFrameWriter.html" target="_blank">Scala</a>): `mode`, `option`, `parquet`, `format`, `saveAsTable`
- StructType (<a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=structtype#pyspark.sql.types.StructType" target="_blank">Python</a>/<a href="http://spark.apache.org/docs/latest/api/scala/org/apache/spark/sql/types/StructType.html" target="_blank" target="_blank">Scala</a>): `toDDL`

##### Spark Types
- Types (<a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=types#module-pyspark.sql.types" target="_blank">Python</a>/<a href="http://spark.apache.org/docs/latest/api/scala/org/apache/spark/sql/types/index.html" target="_blank">Scala</a>): `ArrayType`, `DoubleType`, `IntegerType`, `LongType`, `StringType`, `StructType`, `StructField`

### Read from CSV files
Read from CSV with DataFrameReader's `csv` method and the following options:

Tab separator, use first line as header, infer schema

In [32]:
csvPath = "gs://info323-ya45-spring2023/notebooks/jupyter/2015-summary.csv"

In [33]:
df = (spark.read
  .option("sep", "\t")
  .option("header", True)
  .option("inferSchema", True)
  .csv(csvPath))

In [34]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count: string (nullable = true)



In [35]:
df.show(2)

+-------------------------------------------+
|DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count|
+-------------------------------------------+
|                       United States,Rom...|
|                       United States,Cro...|
+-------------------------------------------+
only showing top 2 rows



In [36]:
df.count()

256

In [40]:
usersPath = "gs://info323-ya45-spring2023/notebooks/jupyter/users-500k/"

In [41]:
usersDF = (spark.read
  .option("sep", "\t")
  .option("header", True)
  .option("inferSchema", True)
  .csv(usersPath))

In [42]:
usersDF.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- user_first_touch_timestamp: long (nullable = true)
 |-- email: string (nullable = true)



In [43]:
usersDF.count()

500000

Manually define the schema by creating a `StructType` with column names and data types

In [44]:
from pyspark.sql.types import LongType, StringType, StructType, StructField

userDefinedSchema = StructType([
  StructField("user_id", StringType(), True),  
  StructField("user_first_touch_timestamp", LongType(), True),
  StructField("email", StringType(), True)
])

Read from CSV using this user-defined schema instead of inferring schema

In [45]:
usersDF = (spark.read
  .option("sep", "\t")
  .option("header", True)
  .schema(userDefinedSchema)
  .csv(usersPath))

Alternatively, define the schema using a DDL formatted string.

In [46]:
DDLSchema = "user_id string, user_first_touch_timestamp long, email string"

usersDF = (spark.read
  .option("sep", "\t")
  .option("header", True)
  .schema(DDLSchema)
  .csv(usersPath))

In [48]:
outputPath = "gs://info323-ya45-spring2023/notebooks/jupyter/output/users-500k"

In [49]:
usersDF.write.option("header", True).option("delimiter", "\t").csv(outputPath)

### Read from JSON files

Read from JSON with DataFrameReader's `json` method and the infer schema option

In [50]:
jsonPath = "gs://info323-ya45-spring2023/notebooks/jupyter/2015-summary.json"

In [53]:
jdf = (spark.read
  .option("inferSchema", True)
  .json(jsonPath))

In [54]:
jdf.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [55]:
jdf.show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 10 rows



In [4]:
jFolderPath = "gs://info323-ya45-spring2023/notebooks/jupyter/events-500k/"

In [57]:
eventsDF = (spark.read
  .option("inferSchema", True)
  .json(jFolderPath))

In [58]:
eventsDF.printSchema()

root
 |-- device: string (nullable = true)
 |-- ecommerce: struct (nullable = true)
 |    |-- purchase_revenue_in_usd: double (nullable = true)
 |    |-- total_item_quantity: long (nullable = true)
 |    |-- unique_items: long (nullable = true)
 |-- event_name: string (nullable = true)
 |-- event_previous_timestamp: long (nullable = true)
 |-- event_timestamp: long (nullable = true)
 |-- geo: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- state: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- coupon: string (nullable = true)
 |    |    |-- item_id: string (nullable = true)
 |    |    |-- item_name: string (nullable = true)
 |    |    |-- item_revenue_in_usd: double (nullable = true)
 |    |    |-- price_in_usd: double (nullable = true)
 |    |    |-- quantity: long (nullable = true)
 |-- traffic_source: string (nullable = true)
 |-- user_first_touch_timestamp: long (nullable = true)

In [59]:
eventsDF.count()

500000

In [2]:
jOutputPath = "gs://info323-ya45-spring2023/notebooks/jupyter/output/events-500k/"

In [61]:
eventsDF.write.json(jOutputPath)

Read data faster by creating a `StructType` with the schema names and data types

In [5]:
from pyspark.sql.types import ArrayType, DoubleType, IntegerType, LongType, StringType, StructType, StructField

userDefinedSchema = StructType([
  StructField("device", StringType(), True),  
  StructField("ecommerce", StructType([
    StructField("purchaseRevenue", DoubleType(), True),
    StructField("total_item_quantity", LongType(), True),
    StructField("unique_items", LongType(), True)
  ]), True),
  StructField("event_name", StringType(), True),
  StructField("event_previous_timestamp", LongType(), True),
  StructField("event_timestamp", LongType(), True),
  StructField("geo", StructType([
    StructField("city", StringType(), True),
    StructField("state", StringType(), True)
  ]), True),
  StructField("items", ArrayType(
    StructType([
      StructField("coupon", StringType(), True),
      StructField("item_id", StringType(), True),
      StructField("item_name", StringType(), True),
      StructField("item_revenue_in_usd", DoubleType(), True),
      StructField("price_in_usd", DoubleType(), True),
      StructField("quantity", LongType(), True)
    ])
  ), True),
  StructField("traffic_source", StringType(), True),
  StructField("user_first_touch_timestamp", LongType(), True),
  StructField("user_id", StringType(), True)
])

eventsDF = (spark.read
  .schema(userDefinedSchema)
  .json(jFolderPath))

In [6]:
DDLSchema = "`device` STRING,`ecommerce` STRUCT<`purchase_revenue_in_usd`: DOUBLE, `total_item_quantity`: BIGINT, `unique_items`: BIGINT>,`event_name` STRING,`event_previous_timestamp` BIGINT,`event_timestamp` BIGINT,`geo` STRUCT<`city`: STRING, `state`: STRING>,`items` ARRAY<STRUCT<`coupon`: STRING, `item_id`: STRING, `item_name`: STRING, `item_revenue_in_usd`: DOUBLE, `price_in_usd`: DOUBLE, `quantity`: BIGINT>>,`traffic_source` STRING,`user_first_touch_timestamp` BIGINT,`user_id` STRING"

eventsDF = (spark.read
  .schema(DDLSchema)
  .json(jFolderPath))

In [7]:
eventsDF.show(3)

+-------+------------------+----------+------------------------+----------------+--------------------+-----+--------------+--------------------------+-----------------+
| device|         ecommerce|event_name|event_previous_timestamp| event_timestamp|                 geo|items|traffic_source|user_first_touch_timestamp|          user_id|
+-------+------------------+----------+------------------------+----------------+--------------------+-----+--------------+--------------------------+-----------------+
|Android|{null, null, null}|mattresses|        1593445137069608|1593445139973236|      {Lakeland, FL}|   []|        google|          1593445100860131|UA000000106062296|
|Android|{null, null, null}|  warranty|        1593448606629109|1593448809022141|     {Rock Hill, SC}|   []|     instagram|          1593448606629109|UA000000106082500|
|Android|{null, null, null}|mattresses|        1593461473732425|1593461479624958|{Stone Mountain, GA}|   []|        google|          1593460845947854|UA000

### Write DataFrames to files

Write `usersDF` to parquet with DataFrameWriter's `parquet` method and the following configurations:

In [10]:
workingDir = "gs://info323-ya45-spring2023/notebooks/jupyter/output"

In [66]:
usersOutputPath = workingDir + "/users.parquet"

(usersDF.write
  .option("compression", "snappy")
  .mode("overwrite")
  .parquet(usersOutputPath)
)

In [8]:
eventsDF.show(3)

+-------+------------------+----------+------------------------+----------------+--------------------+-----+--------------+--------------------------+-----------------+
| device|         ecommerce|event_name|event_previous_timestamp| event_timestamp|                 geo|items|traffic_source|user_first_touch_timestamp|          user_id|
+-------+------------------+----------+------------------------+----------------+--------------------+-----+--------------+--------------------------+-----------------+
|Android|{null, null, null}|mattresses|        1593445137069608|1593445139973236|      {Lakeland, FL}|   []|        google|          1593445100860131|UA000000106062296|
|Android|{null, null, null}|  warranty|        1593448606629109|1593448809022141|     {Rock Hill, SC}|   []|     instagram|          1593448606629109|UA000000106082500|
|Android|{null, null, null}|mattresses|        1593461473732425|1593461479624958|{Stone Mountain, GA}|   []|        google|          1593460845947854|UA000

In [11]:
eventsOutputPath = workingDir + "/events.parquet"

(eventsDF.write
  .option("compression", "snappy")
  .mode("overwrite")
  .parquet(eventsOutputPath)
)